<a href="https://colab.research.google.com/github/joseanu/PMTools/blob/main/mail_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install 'google-api-python-client==1.7.2'
!pip install 'google-auth==1.8.0'
!pip install 'google-auth-httplib2==0.0.3'
!pip install 'google-auth-oauthlib==0.4.1'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google_auth_httplib2 import AuthorizedHttp
from googleapiclient.errors import HttpError
import google.auth.exceptions
from googleapiclient.discovery import build
import base64
import csv
import httplib2

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://mail.google.com/']
our_email = 'joseanu@gmail.com'

In [ ]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('/gdrive/My Drive/token.json'):
    try:
        creds = Credentials.from_authorized_user_file('/gdrive/My Drive/token.json', SCOPES)
        creds.refresh(Request())
    except google.auth.exceptions.RefreshError as error:
        # if refresh token fails, reset creds to none.
        creds = None
        print(f'An error occurred: {error}')
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '/gdrive/My Drive/client_secret.json', SCOPES)
        # creds = flow.run_local_server(port=0)
        creds = flow.run_console()
    # Save the credentials for the next run
    with open('/gdrive/My Drive/token.json', 'w') as token:
        token.write(creds.to_json())


In [ ]:
http = AuthorizedHttp(creds, http=httplib2.Http(timeout=120))
service = build("gmail", "v1", http=http)

  

In [ ]:
# Set the label names you want to search for
label_names = [
    'RED'
]  # Replace with your desired label IDs

In [ ]:
def get_label_name_dict(service):
    labels = service.users().labels().list(userId='me').execute().get('labels', [])
    label_name_dict = {label['id']: label['name'] for label in labels}
    return label_name_dict


def get_email_body(payload):
    if 'parts' in payload:
        parts = payload['parts']
        for part in parts:
            if part['mimeType'] == 'text/plain':
                data = part['body'].get('data')
                if data:
                    return base64.urlsafe_b64decode(data).decode('utf-8')
            else:
                body = get_email_body(part)
                if body:
                    return body
    elif payload['mimeType'] == 'text/plain':
        data = payload['body'].get('data')
        if data:
            return base64.urlsafe_b64decode(data).decode('utf-8')
    return None


def save_emails_to_csv(service, label_names, filename):
    try:
        label_names_str = ' OR '.join([f"label:{label_name}" for label_name in label_names])
        query = f"{label_names_str}"
        print(query)

        response = service.users().messages().list(userId='me', q=query).execute()
        messages = []

        if 'messages' in response:
            messages.extend(response['messages'])

        while 'nextPageToken' in response:
            print(f'Fetching more emails...')
            page_token = response['nextPageToken']
            response = service.users().messages().list(userId='me', q=query, pageToken=page_token).execute()
            if 'messages' in response:
                messages.extend(response['messages'])

        print(f'Total emails: {len(messages)}')

        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Subject', 'From', 'Date', 'To', 'Cc', 'Bcc', 'Message-ID', 'In-Reply-To', 'References',
                          'Labels', 'Thread ID', 'Attachments', 'Body']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for message in messages:
                msg = service.users().messages().get(userId='me', id=message['id'], format='full').execute()
                headers = msg['payload']['headers']
                parts = msg['payload'].get('parts')

                attachments = []
                if parts:
                    for part in parts:
                        filename = part.get('filename')
                        if filename:
                            attachments.append(filename)

                email_data = {name: '' for name in fieldnames}
                email_data['Attachments'] = ', '.join(attachments)
                email_data['Thread ID'] = msg.get('threadId', '')

                for header in headers:
                    if header['name'] in fieldnames:
                        email_data[header['name']] = header['value']

                email_data['Body'] = get_email_body(msg['payload'])

                label_name_dict = get_label_name_dict(service)
                email_data['Labels'] = [label_name_dict.get(label_id, label_id) for label_id in msg.get('labelIds', [])]

                writer.writerow(email_data)

    except HttpError as error:
        print(f'An error occurred: {error}')

In [ ]:
save_emails_to_csv(service, label_names, 'emails.csv')

label:RED
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetching more emails...
Fetchi